In [1]:
# Paso 1: Instalar librerías necesarias
!pip install -q openpyxl

In [2]:
# Paso 2: Importar librerías
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from google.colab import files

In [3]:
# Paso 3: Subir archivo
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

Saving datos_gasto_ampliado.xlsx to datos_gasto_ampliado.xlsx


In [4]:
# Paso 4: Cargar y limpiar columnas
df = pd.read_excel(file_name)
df.columns = df.columns.str.strip().str.lower().str.replace(r'\s+', '_', regex=True).str.replace('[^a-z0-9_]', '', regex=True)

In [5]:

# Paso 5: Verifica nombres resultantes (opcional)
print("Columnas procesadas:", df.columns.tolist())


Columnas procesadas: ['lugar_de_origen', 'transporte_en_el_que_viaja', 'comidas_en_la_uni', 'compra_snacks', 'actividades_extra_en_la_uni', 'lleva_almuerzo', 'compra_almuerzo', 'ocupacion', 'edad', 'cursos_en_el_dia', 'desayuno_en_casa', 'compra_desayuno', 'comparte_transporte', 'hecha_o_da_dinero_para_gasolina', 'gasto_total_q']


In [6]:
# Paso 5: Codificar columnas categóricas
label_cols = [
    'lugar_de_origen', 'transporte_en_el_que_viaja', 'compra_snacks',
    'actividades_extra_en_la_uni', 'lleva_almuerzo', 'compra_almuerzo',
    'ocupacion', 'desayuno_en_casa', 'compra_desayuno',
    'comparte_transporte', 'hecha_o_da_dinero_para_gasolina'
]
label_encoders = {}
for col in label_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [ ]:
#print("Cat columns:", cat_cols)
#print("Num columns:", num_cols)
#print("DataFrame columns:", list(X.columns))

In [7]:
# Paso 6: Separar X e y
X = df.drop("gasto_total_q", axis=1)
y = df["gasto_total_q"]

In [8]:
# Paso 7: Escalar X
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
# Paso 8: División de datos
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

In [10]:
# Paso 9: Entrenar modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [11]:
# Paso 10: Evaluar
y_pred = modelo.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Error absoluto medio (MAE): Q{mae:.2f}")
print(f"Coeficiente de determinación (R²): {r2:.2f}")

Error absoluto medio (MAE): Q34.56
Coeficiente de determinación (R²): -2.82


In [12]:
print(df.head())

   lugar_de_origen  transporte_en_el_que_viaja  comidas_en_la_uni  \
0                6                           1                  2   
1                3                           3                  1   
2                2                           1                  0   
3                1                           2                  2   
4                5                           0                  1   

   compra_snacks  actividades_extra_en_la_uni  lleva_almuerzo  \
0              1                            1               0   
1              0                            0               1   
2              1                            1               1   
3              0                            0               0   
4              1                            1               0   

   compra_almuerzo  ocupacion  edad  cursos_en_el_dia  desayuno_en_casa  \
0                1          1    20                 3                 1   
1                0          2    23         

In [13]:
# Paso 11: Predicción ejemplo
nuevo = pd.DataFrame([{
    "lugar_de_origen": label_encoders["lugar_de_origen"].transform(["Sansare"])[0],
    "transporte_en_el_que_viaja": label_encoders["transporte_en_el_que_viaja"].transform(["Moto"])[0],
    "comidas_en_la_uni": 2,
    "compra_snacks": label_encoders["compra_snacks"].transform(["Si"])[0],
    "actividades_extra_en_la_uni": label_encoders["actividades_extra_en_la_uni"].transform(["No"])[0],
    "lleva_almuerzo": label_encoders["lleva_almuerzo"].transform(["No"])[0],
    "compra_almuerzo": label_encoders["compra_almuerzo"].transform(["Si"])[0],
    "ocupacion": label_encoders["ocupacion"].transform(["Ambas"])[0],
    "edad": 21,
    "cursos_en_el_dia": 4,
    "desayuno_en_casa": label_encoders["desayuno_en_casa"].transform(["No"])[0],
    "compra_desayuno": label_encoders["compra_desayuno"].transform(["Si"])[0],
    "comparte_transporte": label_encoders["comparte_transporte"].transform(["Si"])[0],
    "hecha_o_da_dinero_para_gasolina": label_encoders["hecha_o_da_dinero_para_gasolina"].transform(["Si"])[0],
}])

nuevo_scaled = scaler.transform(nuevo)
prediccion = modelo.predict(nuevo_scaled)
print(f"Gasto estimado: Q{prediccion[0]:.2f}")

Gasto estimado: Q76.20


In [14]:
# --- Exportar modelo y componentes para usarlos en una app ---
import joblib

# Guardar modelo, escalador y codificadores
joblib.dump(modelo, "expenses_model.pkl")
joblib.dump(scaler, "expenses_scaler.pkl")
joblib.dump(label_encoders, "expenses_label_encoders.pkl")

# Descargar los archivos directamente en Colab
from google.colab import files
files.download("expenses_model.pkl")
files.download("expenses_scaler.pkl")
files.download("expenses_label_encoders.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>